# Experiment 1: Clustering on the "curated" dataset


In [1]:
import pandas as pd
import os
import plotly.express as px
from sklearn.metrics.cluster import v_measure_score, adjusted_rand_score
from sklearn.metrics import normalized_mutual_info_score
import numpy as np
import statistics
import openpyxl

In [2]:
from Experiment_1 import run
#run("CamemBERT")
#run("FlauBERT")

C:\Users\stefa\anaconda3\envs\Thesis\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


 Camembert, with the words: ['Avocat', 'Bien', 'Bureau', 'Faculté', 'Filer', 'Glace', 'Souris', 'Supporter', 'Tirer', 'Tour', 'Vol']
 ------------------------- 



Running Avocat with Camembert. Word 1 out of 11
84 left of 107 original sentences
Optimal number of clusters: 3
Score for each sense
avocado    100.000000
lawyer      64.285714
Name: default, dtype: float64
Overall score 76.19047619047619
Removed 0 clusters, which removed 0 points
	 Running experiment with automatic download
524 left of 605 sentences
Optimal number of clusters: 3
Removed 0 clusters
Score for each sense
avocado    100.000000
lawyer      60.714286
Name: default, dtype: float64
Overall score 73.80952380952381


Running Bien with Camembert. Word 2 out of 11
98 left of 99 original sentences
Optimal number of clusters: 4
Score for each sense
good          86.206897
property      69.387755
wellbeing    100.000000
Name: default, dtype: float64
Overall score 80.61224489795919
Removed 0 clusters, which removed 0 point

The second part of this notebook only works if the first part has been ran with both 'Flaubert' and 'Camembert' as model names.

In [3]:
model_name = "Flaubert" # "Camembert" or "Flaubert"

In [5]:
weighted_scores = {} # our original metric
overall_scores = {} # the percentage of correct predictions across all senses
all_in_one_cluster_overall_scores = {} # the percentage of correct predictions if no clustering would take place
amount_senses = {} # the amount of different senses
v_measure_scores = {} # the v-measure score
# We also calculate a F1 score in a later code block
rand_scores = {}
adjusted_mutual_info_score = {}

for filename in os.listdir(f"{model_name}/Experiment_1/Curated/"):
    if filename.endswith(".csv"):
        df = pd.read_csv(f"{model_name}/Experiment_1/Curated/" + filename, sep=";", encoding="utf-8", header=0)
        target_word = df["source"][1]
        print(target_word)
        # We calculate the amount of different senses
        senses = len(df["sense"].unique())
        print("Number of senses:", senses)
        percentage_default = (df["default"].sum() / len(df))
        # We also calculate this separately for each "sense"
        percentage_default_mean = df.groupby("sense")["default"].mean()
        # We calculate a harmonic mean, as it is more representative of the overall performance
        harmonic_mean = statistics.harmonic_mean(df.groupby("sense")["default"].mean())
        # We want the mean score across all senses, as it does not mean a lot if a program can correctly define one big cluster containing most of the data and fail at all other senses.
        percentage_weighted = percentage_default_mean.mean()
        v_measure = v_measure_score(df["sense"], df["cluster"])
        rand = adjusted_rand_score(df["sense"], df["cluster"])
        mutual_info = normalized_mutual_info_score(df["sense"], df["cluster"])


        print("Score for each", percentage_default_mean*100)
        print("Overall score", percentage_default*100)
        print("Weighted score", percentage_weighted*100)
        print("Harmonic mean", harmonic_mean*100)
        print("V-measure score", v_measure*100)
        print("Adjusted Rand score", rand*100)
        print("Adjusted mutual info score", mutual_info*100)
        weighted_scores[target_word] = percentage_weighted
        overall_scores[target_word] = percentage_default
        amount_senses[target_word] = senses
        v_measure_scores[target_word] = v_measure
        rand_scores[target_word] = rand
        adjusted_mutual_info_score[target_word] = mutual_info
        # We also calculate the score if no clustering would take place
        default = df["sense"].value_counts().idxmax()
        
        # We calculate the overall score: this is the percentage of this sense in the total dataframe
        all_in_one_cluster_score = (df["sense"] == default).sum() / len(df)
        all_in_one_cluster_overall_scores[target_word] = all_in_one_cluster_score
        print("All-in-one score", all_in_one_cluster_score*100, "\n\n")

Avocat
Number of senses: 2
Score for each sense
avocado    82.142857
lawyer     94.642857
Name: default, dtype: float64
Overall score 90.47619047619048
Weighted score 88.39285714285714
Harmonic mean 87.95093795093794
V-measure score 70.20502716093677
Adjusted Rand score 80.36426589295034
Adjusted mutual info score 70.20502716093677
All-in-one score 66.66666666666666 


Bien
Number of senses: 3
Score for each sense
good          96.551724
property     100.000000
wellbeing    100.000000
Name: default, dtype: float64
Overall score 98.9795918367347
Weighted score 98.85057471264368
Harmonic mean 98.82352941176471
V-measure score 95.41014187403705
Adjusted Rand score 96.54410605877042
Adjusted mutual info score 95.41014187403705
All-in-one score 50.0 


Bureau
Number of senses: 3
Score for each sense
cabinet      0.000000
desk       100.000000
office      40.740741
Name: default, dtype: float64
Overall score 55.33980582524271
Weighted score 46.91358024691358
Harmonic mean 0
V-measure score 3

In [4]:

for filename in os.listdir(f"{model_name}/Experiment_1/Curated/"):
    if filename.endswith(".csv"):
        df = pd.read_csv(f"{model_name}/Experiment_1/Curated/" + filename, sep=";", encoding="utf-8", header=0)
        target_word = df["source"][1]
        print(target_word)
        # We calculate the amount of different senses
        senses = len(df["sense"].unique())
        print("Number of senses:", senses)
        mutual_info = normalized_mutual_info_score(df["sense"], df["cluster"])
        adjusted_mutual_info_score[target_word] = mutual_info
        print("Adjusted mutual info score", mutual_info*100)

Avocat
Number of senses: 2
Adjusted mutual info score 70.20502716093677
Bien
Number of senses: 3
Adjusted mutual info score 95.41014187403705
Bureau
Number of senses: 3
Adjusted mutual info score 36.06389452968297
Faculté
Number of senses: 3
Adjusted mutual info score 79.88668250065365
Filer
Number of senses: 3
Adjusted mutual info score 88.18594500715287
Glace
Number of senses: 5
Adjusted mutual info score 46.43790540203866
Souris
Number of senses: 3
Adjusted mutual info score 53.96005727595151
Supporter
Number of senses: 3
Adjusted mutual info score 34.922709572364106
Tirer
Number of senses: 2
Adjusted mutual info score 17.379107256753016
Tour
Number of senses: 5
Adjusted mutual info score 72.56160915561088
Vol
Number of senses: 2
Adjusted mutual info score 80.23925458634227


# Calculating true F1 score

To have a comparable measure of cluster quality, we will calculate F1 scores
they consist of Precision and Recall scores, and is calculated by taking their harmonic mean

Precision is the amount of correct instances of a sense in its biggest cluster, divided by the number of points in that cluster
Recall is the amount of correct instances of a sense in its biggest cluster, divided by the number of points in the sense

we save these values in a dictionary of dictionaries for each target word, which in turn contain precision and recall scores for each sense


Using skmetrics f1 does not work (gives values of 0 where it should not), so we will calculate it ourselves

In [23]:
# We create a dictionary of dictionaries to save, for each sense, the ratings of its precision and recall
all_scores = {}
all_lengths = {}
for filename in os.listdir(f"{model_name}/Experiment_1/Curated/"):
    if filename.endswith(".csv"):
        df_F1 = pd.read_csv(f"{model_name}/Experiment_1/Curated/" + filename, sep=";", encoding="utf-8", header=0)

        target_word = df_F1["source"][1]
        print("\n\n", target_word)
        total_df_length = len(df_F1)
        # We calculate the amount of different senses
        senses = len(df_F1["sense"].unique())
        print("Number of senses:", senses)
        target_word_dictionary = {}
        target_word_lengths = {}
        target_word_lengths["total"] = total_df_length
        for sense in df_F1["sense"].unique():
            scores = {}
            sense_df = df_F1[df_F1["sense"] == sense]
                    #precision: number of correct instances in a cluster / total cluster size
                    # We calculate the amount of "default" instances
            sense_len = len(sense_df)
        
            
            correct_df = sense_df[sense_df["default"] == True]
            correct_len = len(correct_df)
            if correct_len > 0:
                correct_cluster = correct_df["cluster"].iloc[0]
            else:
                correct_cluster = "No cluster"
                
            cluster_len = len(df_F1[df_F1["cluster"] == correct_cluster])
            print("\nsense:", sense)
            print("instances of sense:", sense_len)
            print("amount of correct instances:", correct_len)
            print("correct cluster number:", correct_cluster)
            print("total amount of points in biggest cluster:", cluster_len)
            
            if cluster_len > 0:  
                precision = correct_len / cluster_len
            else:
                precision = 0
                print("precision zero")
            recall = correct_len / sense_len
            print("precision:", precision)
            print("recall", recall)
            
            F1 = statistics.harmonic_mean((precision, recall))
            print("F1", F1)
            
            scores["precision"] = precision
            scores["recall"] = recall
            scores["F1"] = F1
            target_word_dictionary[sense] = scores
            target_word_lengths[sense] = sense_len
        all_scores[target_word] = target_word_dictionary
        all_lengths[target_word] = target_word_lengths
            



 Avocat
Number of senses: 2

sense: lawyer
instances of sense: 56
amount of correct instances: 53
correct cluster number: 1
total amount of points in biggest cluster: 54
precision: 0.9814814814814815
recall 0.9464285714285714
F1 0.9636363636363636

sense: avocado
instances of sense: 28
amount of correct instances: 23
correct cluster number: 2
total amount of points in biggest cluster: 23
precision: 1.0
recall 0.8214285714285714
F1 0.9019607843137255


 Bien
Number of senses: 3

sense: property
instances of sense: 49
amount of correct instances: 49
correct cluster number: 1
total amount of points in biggest cluster: 50
precision: 0.98
recall 1.0
F1 0.9898989898989898

sense: wellbeing
instances of sense: 20
amount of correct instances: 20
correct cluster number: 2
total amount of points in biggest cluster: 20
precision: 1.0
recall 1.0
F1 1.0

sense: good
instances of sense: 29
amount of correct instances: 28
correct cluster number: 3
total amount of points in biggest cluster: 28
preci

In [24]:
# Total F1: weighted average of the F1 scores of each gold standard sense
# We calculate the weight for each sense of the dataframe (the amount of instances of this sense divided by the total amount of instances)
# We make a dictionary F1_scores, in which each target word is associated with its F1 score
F1_scores = {}
for target_word in all_scores:
    F1 = 0
    weight = 0
    print("\n\n", target_word)
    for sense in all_scores[target_word]:
        score = all_scores[target_word][sense]["F1"]
        print(sense)
        print("score:", score)
        
        len_sense = all_lengths[target_word][sense]
        print("length of sense:", len_sense)
        
        len_total = all_lengths[target_word]["total"]
        print("total length:", len_total)
        added_weight = len_sense / len_total
        weight += added_weight
        print("added weight:", added_weight, "for a total of", weight)
        F1 += score * (len_sense / len_total)
        print("F1 score after calc:", F1)
    F1_scores[target_word] = F1





 Avocat
lawyer
score: 0.9636363636363636
length of sense: 56
total length: 84
added weight: 0.6666666666666666 for a total of 0.6666666666666666
F1 score after calc: 0.6424242424242423
avocado
score: 0.9019607843137255
length of sense: 28
total length: 84
added weight: 0.3333333333333333 for a total of 1.0
F1 score after calc: 0.9430778371954842


 Bien
property
score: 0.9898989898989898
length of sense: 49
total length: 98
added weight: 0.5 for a total of 0.5
F1 score after calc: 0.4949494949494949
wellbeing
score: 1.0
length of sense: 20
total length: 98
added weight: 0.20408163265306123 for a total of 0.7040816326530612
F1 score after calc: 0.6990311276025561
good
score: 0.9824561403508772
length of sense: 29
total length: 98
added weight: 0.29591836734693877 for a total of 1.0
F1 score after calc: 0.9897579446451628


 Bureau
desk
score: 0.6862745098039216
length of sense: 35
total length: 103
added weight: 0.33980582524271846 for a total of 0.33980582524271846
F1 score after cal

In [25]:
# We will also calculate an F1 score for each word when no clustering takes place
# We will make a copy of each dataframe and set the "cluster" column to 0 for all rows
# We create a dictionary of dictionaries to save, for each sense, the ratings of its precision and recall
all_scores_no_cluster = {}
all_lengths_no_cluster = {}
for filename in os.listdir(f"{model_name}/Experiment_1/Curated/"):
    if filename.endswith(".csv"):
        df_F1 = pd.read_csv(f"{model_name}/Experiment_1/Curated/" + filename, sep=";", encoding="utf-8", header=0)

        target_word = df_F1["source"][1]
        # We undo clustering: we set all clusters to 0
        # The "default" (i.e. "correct" category is the most frequent sense)
        df_F1["cluster"] = 0
        most_freq_sense = df_F1["sense"].value_counts().idxmax()
        df_F1["default"] = [1 if sense == most_freq_sense else 0 for sense in df_F1["sense"]]
        print("\n\n", target_word)
        total_df_length = len(df_F1)
        # We calculate the amount of different senses
        senses = len(df_F1["sense"].unique())
        print("Number of senses:", senses)
        target_word_dictionary = {}
        target_word_lengths = {}
        target_word_lengths["total"] = total_df_length
        for sense in df_F1["sense"].unique():
            scores = {}
            sense_df = df_F1[df_F1["sense"] == sense]
                    #precision: number of correct instances in a cluster / total cluster size
                    # We calculate the amount of "default" instances
            sense_len = len(sense_df)
        
            
            correct_df = sense_df[sense_df["default"] == True]
            correct_len = len(correct_df)
            if correct_len > 0:
                correct_cluster = correct_df["cluster"].iloc[0]
            else:
                correct_cluster = "No cluster"
                
            cluster_len = len(df_F1[df_F1["cluster"] == correct_cluster])
            print("\nsense:", sense)
            print("instances of sense:", sense_len)
            print("amount of correct instances:", correct_len)
            print("correct cluster number:", correct_cluster)
            print("total amount of points in biggest cluster:", cluster_len)
            
            if cluster_len > 0:  
                precision = correct_len / cluster_len
            else:
                precision = 0
                print("precision zero")
            recall = correct_len / sense_len
            print("precision:", precision)
            print("recall", recall)
            
            F1 = statistics.harmonic_mean((precision, recall))
            print("F1", F1)
            
            scores["precision"] = precision
            scores["recall"] = recall
            scores["F1"] = F1
            target_word_dictionary[sense] = scores
            target_word_lengths[sense] = sense_len
        all_scores_no_cluster[target_word] = target_word_dictionary
        all_lengths_no_cluster[target_word] = target_word_lengths
# Total F1: weighted average of the F1 scores of each gold standard sense
# We calculate the weight for each sense of the dataframe (the amount of instances of this sense divided by the total amount of instances)
# We make a dictionary F1_scores, in which each target word is associated with its F1 score
F1_scores_no_cluster = {}
for target_word in all_scores_no_cluster:
    F1 = 0
    weight = 0
    print("\n\n", target_word)
    for sense in all_scores_no_cluster[target_word]:
        score = all_scores_no_cluster[target_word][sense]["F1"]
        print(sense)
        print("score:", score)
        
        len_sense = all_lengths_no_cluster[target_word][sense]
        print("length of sense:", len_sense)
        
        len_total = all_lengths_no_cluster[target_word]["total"]
        print("total length:", len_total)
        added_weight = len_sense / len_total
        weight += added_weight
        print("added weight:", added_weight, "for a total of", weight)
        F1 += score * (len_sense / len_total)
        print("F1 score after calc:", F1)
    F1_scores_no_cluster[target_word] = F1





 Avocat
Number of senses: 2

sense: lawyer
instances of sense: 56
amount of correct instances: 56
correct cluster number: 0
total amount of points in biggest cluster: 84
precision: 0.6666666666666666
recall 1.0
F1 0.8

sense: avocado
instances of sense: 28
amount of correct instances: 0
correct cluster number: No cluster
total amount of points in biggest cluster: 0
precision zero
precision: 0
recall 0.0
F1 0


 Bien
Number of senses: 3

sense: property
instances of sense: 49
amount of correct instances: 49
correct cluster number: 0
total amount of points in biggest cluster: 98
precision: 0.5
recall 1.0
F1 0.6666666666666666

sense: wellbeing
instances of sense: 20
amount of correct instances: 0
correct cluster number: No cluster
total amount of points in biggest cluster: 0
precision zero
precision: 0
recall 0.0
F1 0

sense: good
instances of sense: 29
amount of correct instances: 0
correct cluster number: No cluster
total amount of points in biggest cluster: 0
precision zero
precisio

In [26]:
Nouns = ["Avocat", "Bien", "Bureau", "Faculté", "Glace", "Souris", "Tour", "Vol"]
Verbs = ["Filer", "Supporter", "Tirer"]

nouns_average_weighted = np.mean([weighted_scores[noun] for noun in Nouns])
nouns_average_overall = np.mean([overall_scores[noun] for noun in Nouns])
nouns_average_all_in_one = np.mean([all_in_one_cluster_overall_scores[noun] for noun in Nouns])
nouns_average_v_measure = np.mean([v_measure_scores[noun] for noun in Nouns])
nouns_average_F1 = np.mean([F1_scores[noun] for noun in Nouns])
nouns_average_rand = np.mean([rand_scores[noun] for noun in Nouns])
nouns_average_F1_no_cluster = np.mean([F1_scores_no_cluster[noun] for noun in Nouns])


verbs_average_weighted = np.mean([weighted_scores[verb] for verb in Verbs])
verbs_average_overall = np.mean([overall_scores[verb] for verb in Verbs])
verbs_average_all_in_one = np.mean([all_in_one_cluster_overall_scores[verb] for verb in Verbs])
verbs_average_v_measure = np.mean([v_measure_scores[verb] for verb in Verbs])
verbs_average_rand = np.mean([rand_scores[verb] for verb in Verbs])
verbs_average_F1 = np.mean([F1_scores[verb] for verb in Verbs])
verbs_average_F1_no_cluster = np.mean([F1_scores_no_cluster[verb] for verb in Verbs])

print("Nouns average of weighted scores:", nouns_average_weighted)
print("Nouns average of overall scores:", nouns_average_overall)
print("Nouns average of all in one cluster scores:", nouns_average_all_in_one)
print("Nouns average of v-measure cluster score:", nouns_average_v_measure)
print("Nouns average of Adjusted Rand scores:", nouns_average_rand)
print("Nouns average of F1 scores:", nouns_average_F1)
print("Nouns average of F1 scores when no clustering takes place:", nouns_average_F1_no_cluster)
print("\n\n")
print("Verbs average of weighted scores:", verbs_average_weighted)
print("Verbs average of overall scores:", verbs_average_overall)
print("Verbs average of all in one cluster scores:", verbs_average_all_in_one)
print("Verbs average of v-measure cluster score:", verbs_average_v_measure)
print("Verbs average of F1 scores:", verbs_average_F1)
print("Verbs average of Adjusted Rand scores:", verbs_average_rand)
print("Verbs average of F1 scores when no clustering takes place:", verbs_average_F1_no_cluster)


Nouns average of weighted scores: 0.7374825239445986
Nouns average of overall scores: 0.7493163916379987
Nouns average of all in one cluster scores: 0.5149002922851882
Nouns average of v-measure cluster score: 0.6684557156065671
Nouns average of Adjusted Rand scores: 0.5900182693692801
Nouns average of F1 scores: 0.7623989627197997
Nouns average of F1 scores when no clustering takes place: 0.3574206110459456



Verbs average of weighted scores: 0.6365604206029738
Verbs average of overall scores: 0.5866550116550117
Verbs average of all in one cluster scores: 0.5700709013209013
Verbs average of v-measure cluster score: 0.46829253945423327
Verbs average of F1 scores: 0.6632169624936243
Verbs average of Adjusted Rand scores: 0.31882184711744016
Verbs average of F1 scores when no clustering takes place: 0.41897442708157473


In [27]:
# We create a dataframe from the dictionaries to eventually save the results to a csv file
df = pd.DataFrame.from_dict(weighted_scores, orient="index", columns=["Weighted"])
df["Overall"] = overall_scores.values()
df["V_measure"] = v_measure_scores.values()
df["Rand"] = rand_scores.values()
df["F1"] = [F1_scores[word] for word in df.index]

df["Type"] = ["Noun" if word in Nouns else "Verb" for word in df.index]
df["Amount_senses"] = amount_senses.values()
# We add a "F1" column to the dataframe

# We also create a separate file for the all in one cluster scores
df_all_in_one = pd.DataFrame.from_dict(all_in_one_cluster_overall_scores, orient="index", columns=["Overall_no_cluster"])
df_all_in_one["F1_no_cluster"] = [F1_scores_no_cluster[word] for word in df_all_in_one.index]
df_all_in_one["Type"] = ["Noun" if word in Nouns else "Verb" for word in df_all_in_one.index]

In [28]:
# We add a row 'Average' to the dataframes
df_with_average = df.copy()
df_with_average.loc["Average_nouns"] = [nouns_average_weighted, nouns_average_overall, nouns_average_v_measure, nouns_average_rand, nouns_average_F1, "", ""]
df_with_average.loc["Average_verbs"] = [verbs_average_weighted, verbs_average_overall, verbs_average_v_measure, verbs_average_rand, verbs_average_F1, "", ""]
df_with_average.loc["Global_average"] = [df["Weighted"].mean(), df["Overall"].mean(), df["V_measure"].mean(), df["Rand"].mean(), df["F1"].mean(), "", ""] #use the og file, no account of averages


# As we don't make a copy, we have to calculate the mean before adding the other averages!
df_all_in_one.loc["Global_average"] = [df_all_in_one["Overall_no_cluster"].mean(), df_all_in_one["F1_no_cluster"].mean(), "",] 
df_all_in_one.loc["Average_nouns"] = [nouns_average_all_in_one, nouns_average_F1_no_cluster,""]
df_all_in_one.loc["Average_verbs"] = [verbs_average_all_in_one, verbs_average_F1_no_cluster,""]


In [29]:
fig_zegt_weinig = px.scatter(df, x="Weighted", y="Overall", text=df.index, color="Type", title="Scores for each word, op een manier die weinig zegt eigenlijk",template="plotly_white")
fig_zegt_weinig.show()
fig_zegt_weinig = px.scatter(df, x="Weighted", y="Overall", text=df.index, color="Type", title="Scores for each word, op een manier die weinig zegt eigenlijk",template="plotly_white")
fig = px.scatter(df, x="V_measure", y="F1", text=df.index, color="Type", title="Scores for each word",template="plotly_white")
fig.show()

# Estimating the number of clusters
We will now calculate if the model correctly predicts the number of clusters.
We will count the number of different manually annotated senses and compare this to the amount of generated clusters.

In [30]:
correct = []
close = [] # One cluster off, often the result of different granularity. Is it ok to report like this or do we have to be more strict?
incorrect = []
differences = {}
for filename in os.listdir(f"{model_name}/Experiment_1/Curated/"):
    if filename.endswith(".csv"):
        df = pd.read_csv(f"{model_name}/Experiment_1/Curated/" + filename, sep=";", encoding="utf-8", header=0)
        target_word = df["source"][1]
        print(target_word)
        # We count the number of different senses
        senses = len(df["sense"].unique())
        # We count the number of clusters
        clusters = len(df["cluster"].unique())
        print("Number of senses:", senses)
        print("Number of clusters:", clusters)
        if senses == clusters:
            print("Correct number of clusters")
            difference = 0
            correct.append(target_word)
        elif senses > clusters:
            difference = senses - clusters
            print(difference, "too few clusters")
            if difference == 1:
                close.append(target_word)
            else:
                incorrect.append(target_word)
            difference = -difference
        elif senses < clusters:
            difference = clusters - senses
            print(difference, "too many clusters")
            if difference == 1:
                close.append(target_word)
            else:
                incorrect.append(target_word)
                
        differences[target_word] = difference
        # We calculate the score
        #print("Score:", score)
        #scores[target_word] = score

print("Correct:", correct)
print("Close:", close)
print("Incorrect:", incorrect)
print("Percentage approach:", (len(correct) + len(close)) / ((len(correct) + len(close) + len(incorrect))))

Avocat
Number of senses: 2
Number of clusters: 3
1 too many clusters
Bien
Number of senses: 3
Number of clusters: 3
Correct number of clusters
Bureau
Number of senses: 3
Number of clusters: 2
1 too few clusters
Faculté
Number of senses: 3
Number of clusters: 2
1 too few clusters
Filer
Number of senses: 3
Number of clusters: 4
1 too many clusters
Glace
Number of senses: 5
Number of clusters: 4
1 too few clusters
Souris
Number of senses: 3
Number of clusters: 9
6 too many clusters
Supporter
Number of senses: 3
Number of clusters: 10
7 too many clusters
Tirer
Number of senses: 2
Number of clusters: 3
1 too many clusters
Tour
Number of senses: 5
Number of clusters: 8
3 too many clusters
Vol
Number of senses: 2
Number of clusters: 3
1 too many clusters
Correct: ['Bien']
Close: ['Avocat', 'Bureau', 'Faculté', 'Filer', 'Glace', 'Tirer', 'Vol']
Incorrect: ['Souris', 'Supporter', 'Tour']
Percentage approach: 0.7272727272727273


**Results for Camembert:**
Correct: ['Filer', 'Glace']
Close: ['Avocat', 'Bien', 'Bureau', 'Faculté', 'Tirer', 'Tour', 'Vol']
Incorrect: ['Souris', 'Supporter']
Percentage approach: 0.8181818181818182

**Results for Flaubert:**
Correct: ['Bien']
Close: ['Avocat', 'Bureau', 'Faculté', 'Filer', 'Glace', 'Tirer', 'Vol']
Incorrect: ['Souris', 'Supporter', 'Tour']
Percentage approach: 0.7272727272727273

In [31]:
# We add a "difference" column to the dataframe
df_with_average["Cluster_Estimation"] = [differences[word] if word in differences else "" for word in df_with_average.index]

In [32]:
# We save these dataframes to a csv file
if not os.path.exists("Results/Experiment_1/Curated/"):
    os.makedirs("Results/Experiment_1/Curated")
df_with_average.to_csv(f"Results/Experiment_1/Curated/{model_name}_scores.csv", sep=";", encoding="utf-8")
fig.write_html(f"Results/Experiment_1/Curated/{model_name}_scores.html")
df_all_in_one.to_csv(f"Results/Experiment_1/Curated/all_in_one_scores.csv", sep=";", encoding="utf-8")

# Comparing the model_names

In [33]:
# We load in both dataframes in the "Results" folder
df_camembert = pd.read_csv("Results/Experiment_1/Curated/Camembert_scores.csv", sep=";", encoding="utf-8", index_col=0)
df_flaubert = pd.read_csv("Results/Experiment_1/Curated/Flaubert_scores.csv", sep=";", encoding="utf-8", index_col=0)

# We add a column to both dataframes to indicate the model
df_camembert["Model"] = "Camembert"
df_flaubert["Model"] = "Flaubert"

# We concatenate both dataframes
df = pd.concat([df_camembert, df_flaubert])

In [34]:
# We split the dataframe into two separate dataframes
df_weighted = df[["Weighted", "Model"]]
df_overall = df[["Overall", "Model"]]
df_v_measure = df[["V_measure", "Model"]]
df_rand = df[["Rand", "Model"]]
df_f1 = df[["F1", "Model"]]

In [35]:
# We transform the dataframes:
# - We make two columns, one for each model
# - We regroup the instances of the same word (index) in one row
df_weighted = df_weighted.pivot(columns="Model")
df_overall = df_overall.pivot(columns="Model")
df_v_measure = df_v_measure.pivot(columns="Model")
df_rand = df_rand.pivot(columns="Model")
df_f1 = df_f1.pivot(columns="Model")

In [36]:
# We save this dataframe to a csv file and to excel
df_weighted.to_csv("Results/Experiment_1/Curated/weighted_scores.csv", sep=";", encoding="utf-8")
df_overall.to_csv("Results/Experiment_1/Curated/overall_scores.csv", sep=";", encoding="utf-8")
df_v_measure.to_csv("Results/Experiment_1/Curated/v_measure_scores.csv", sep=";", encoding="utf-8")
df_rand.to_csv("Results/Experiment_1/Curated/rand_scores.csv", sep=";", encoding="utf-8")
df_f1.to_csv("Results/Experiment_1/Curated/F1_scores.csv", sep=";", encoding="utf-8")

df_weighted.to_excel("Results/Experiment_1/Curated/weighted_scores.xlsx")
df_overall.to_excel("Results/Experiment_1/Curated/overall_scores.xlsx")
df_v_measure.to_excel("Results/Experiment_1/Curated/v_measure_scores.xlsx")
df_rand.to_excel("Results/Experiment_1/Curated/rand_scores.xlsx")
df_f1.to_excel("Results/Experiment_1/Curated/F1_scores.xlsx")


In [37]:
# using openpyxl, we open the Results/weighted_scores.xlsx file
# we put the highest value of each row in bold
wb = openpyxl.load_workbook("Results/Experiment_1/Curated/weighted_scores.xlsx")
ws = wb.active
for row in ws.iter_rows(min_row=4, max_row=ws.max_row, min_col=2, max_col=3):
    max_value = max([cell.value for cell in row])
    for cell in row:
        if cell.value == max_value:
            cell.font = openpyxl.styles.Font(bold=True)
wb.save("Results/Experiment_1/Curated/weighted_scores.xlsx")

# We do the same for the overall scores
wb = openpyxl.load_workbook("Results/Experiment_1/Curated/overall_scores.xlsx")
ws = wb.active
for row in ws.iter_rows(min_row=4, max_row=ws.max_row, min_col=2, max_col=3):
    max_value = max([cell.value for cell in row])
    for cell in row:
        if cell.value == max_value:
            cell.font = openpyxl.styles.Font(bold=True)
wb.save("Results/Experiment_1/Curated/overall_scores.xlsx")

# We do the same for the v measure scores
wb = openpyxl.load_workbook("Results/Experiment_1/Curated/v_measure_scores.xlsx")
ws = wb.active
for row in ws.iter_rows(min_row=4, max_row=ws.max_row, min_col=2, max_col=3):
    max_value = max([cell.value for cell in row])
    for cell in row:
        if cell.value == max_value:
            cell.font = openpyxl.styles.Font(bold=True)
wb.save("Results/Experiment_1/Curated/v_measure_scores.xlsx")

# and the same for f1 scores

# We do the same for the overall scores
wb = openpyxl.load_workbook("Results/Experiment_1/Curated/F1_scores.xlsx")
ws = wb.active
for row in ws.iter_rows(min_row=4, max_row=ws.max_row, min_col=2, max_col=3):
    max_value = max([cell.value for cell in row])
    for cell in row:
        if cell.value == max_value:
            cell.font = openpyxl.styles.Font(bold=True)
wb.save("Results/Experiment_1/Curated/F1_scores.xlsx")

In [38]:
df.to_csv("Results/Experiment_1/Curated/results.csv", sep=";", encoding="utf-8")

In [39]:
# We will make boxplots for the weighted scores of each word
fig_weighted = px.box(df, x="Model" ,y="Weighted", color="Model", title="Weighted scores for each model", template="plotly_white")
fig_weighted.update(layout_showlegend=False)
fig_weighted.show()

fig_overall = px.box(df, x="Model" ,y="Overall", color="Model", title="Overall scores for each model", template="plotly_white")
fig_overall.update(layout_showlegend=False)
fig_overall.show()

fig_v_measure = px.box(df, x="Model" ,y="V_measure", color="Model", title="V measure clustering scores for each model", template="plotly_white")
fig_v_measure.update(layout_showlegend=False)
fig_v_measure.show()

fig_rand = px.box(df, x="Model" ,y="Rand", color="Model", title="Adjusted Rand scores for each model", template="plotly_white")
fig_rand.update(layout_showlegend=False)
fig_rand.show()

fig_F1 = px.box(df, x="Model" ,y="F1", color="Model", title="F1 scores for each model", template="plotly_white")
fig_F1.update(layout_showlegend=False)
fig_F1.show()

As we are more familiar with data visualisation in R, we will use the saved files to create the visualisations in R.

